In [1]:
import pandas as pd
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
SOS_token = 0
EOS_token = 1
UNK_token=2
class Lang:
    def __init__(self,name):
        self.name=name
        self.word2index={}
        self.word2count={}
        self.index2word={0:'sos',1:'eos','unk':2}
        self.n_words=2
    def addSentence(self,sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self,word):
        if word not in self.word2index:
            self.word2index[word]=self.n_words
            self.word2count[word]=1
            self.index2word[self.n_words]=word
            self.n_words+=1
        else:
            self.word2count[word]+=1


In [3]:
encoder_path='/content/drive/MyDrive/NMT_Model/encoder_model.pt'
decoder_path='/content/drive/MyDrive/NMT_Model/decoder_model.pt'
lang_path='/content/drive/MyDrive/NMT_Model/lang_instances.pkl'

In [13]:
import pickle
# Load instances of your custom Lang class from the file
with open(lang_path, 'rb') as file:
    loaded_lang_instances = pickle.load(file)

# Access individual instances by language
english_lang_instance = loaded_lang_instances['english']
hnglish_lang_instance = loaded_lang_instances['henglish']

# Access more languages and instances as needed



In [14]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    #s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()


In [15]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

In [16]:
class GlobalAttention(nn.Module):
    def __init__(self, hidden_size):
        super(GlobalAttention, self).__init__()
        self.Wq = nn.Linear(hidden_size, hidden_size)
        self.Wk = nn.Linear(hidden_size, hidden_size)
        self.Wv = nn.Linear(hidden_size, hidden_size)

    def forward(self, query, keys):
        # Linear transformations
        q = self.Wq(query)
        k = self.Wk(keys)
        v = self.Wv(keys)

        # Calculate attention scores (global attention)
        scores = torch.matmul(q, k.transpose(1, 2))

        # Apply softmax to obtain attention weights
        weights = torch.nn.functional.softmax(scores, dim=-1)

        # Compute the context vector based on attention weights
        context = torch.matmul(weights, v)

        return context, weights

In [17]:
MAX_LENGTH=15

In [18]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size,attention, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention_name=attention
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)
        if self.attention_name == 'GlobalAttention':
            self.attention=GlobalAttention(hidden_size)
        else:
            print('Error choose Correct Attention')



    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions

    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)

        if self.attention_name=='BahdanauAttention':
            context, attn_weights = self.attention(query, encoder_outputs)
        elif self.attention_name=='DotProductAttentionLinear':
            context, attn_weights = self.attention(query, encoder_outputs,encoder_outputs)
        elif self.attention_name=='SelfAttention':
            context, attn_weights = self.attention(query, encoder_outputs,encoder_outputs)
        elif self.attention_name== 'LocationBasedAttention':
            context, attn_weights = self.attention(query, encoder_outputs)
        elif self.attention_name == 'GlobalAttention':
            context, attn_weights = self.attention(query, encoder_outputs)
        else:
            print('Error choose Correct Attention')

        input_gru = torch.cat((embedded, context), dim=2)
        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

In [19]:
def indexesFromSentence(lang, sentence):
    words=sentence.split(' ')
    return [lang.word2index.get(word,UNK_token) for word in words]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

In [20]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn
def evaluateRandomly(encoder, decoder, input_sentence,input_lang,output_lang):
        output_words, _ = evaluate(encoder, decoder, input_sentence, input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        return output_sentence

In [21]:
hidden_size = 128
batch_size = 32

value='GlobalAttention'
encoder = EncoderRNN(english_lang_instance.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, hnglish_lang_instance.n_words,attention=value).to(device)
learning_rate=0.001
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()
# Load the trained model weights
encoder.load_state_dict(torch.load(encoder_path))
decoder.load_state_dict(torch.load(decoder_path))


<All keys matched successfully>

In [36]:
while True:
  input_sentence=input('Enter Sentence in English : ')
  if input_sentence=='Quit':
    print('Bye Bye...')
    break
  #prediction
  encoder.eval()
  decoder.eval()
  predictions=evaluateRandomly(encoder, decoder,input_sentence,english_lang_instance,hnglish_lang_instance)
  print('Output Sentence : ',predictions)
  print()


Enter Sentence in English : How Long it will be raining ?
Output Sentence :  kya aaj baarish horahi hai  ? <EOS>

Enter Sentence in English : play the top 10 songs of 1990
Output Sentence :  1990 ke top 10 songs bajao <EOS>

Enter Sentence in English : 	is it sunny today ?
Output Sentence :  aaj sunny day hai  ? <EOS>

Enter Sentence in English : Quit
Bye Bye...
